In [ ]:
#对各站点的日降水数据汇总成一个大表格
import pandas as pd
import os

def datewind(x, name): 
    def standardformat(x):
        if len(str(x)) == 1:
            x = '0' + str(x)
        return str(x)
    def intstr(x):
        try:
            y = int(x)
            y = str(y)
        except:
            y = str(x)
        finally:
            pass
        return y
    
    yearx = x['年'].map(lambda x: intstr(x))
    monthx = x['月'].map(lambda x: standardformat(intstr(x)))
    dayx = x['日'].map(lambda x: standardformat(intstr(x)))
    Identification = yearx.str.cat(monthx + dayx)

    dataresult = pd.DataFrame({'唯一标示': Identification,
                                name: x.loc[:,'降水量(mm)']})
    dataresult.drop_duplicates()
    return dataresult

In [ ]:
pointsname = pd.read_excel('point.xls', index_col=None, na_values=['NA'])
pointnumber = pointsname.iloc[:,0].tolist()
pointname = pointsname.iloc[:,1].tolist()
pointprovince = pointsname.iloc[:,2].tolist()
numbername = dict(zip(pointnumber, pointname)) #别把turple和dict搞错了
numberprovince = dict(zip(pointnumber, pointprovince))

coastpointnumber = []
coastpointnamelist = []
for point in pointnumber:
    coastpointnumber.append (point)
    coastpointnamelist.append (numbername.get(point))
        
coastnumbername = dict(zip(coastpointnumber,coastpointnamelist))

In [ ]:
data = pd.read_excel(r'广东\广州 59287.xlsx',  index_col=None, na_values=['NA'])
dataresult = pd.DataFrame({'唯一标示': datewind(data, '广州').loc[:, '唯一标示']})
count = 0

for point in coastpointnumber:
    readpath =  numberprovince.get(point) + '/' + str(numbername.get(point)) +' '+ str(point) +'.xlsx'
    if os.path.exists(readpath) == True:
        partdata = pd.read_excel(readpath, 'result', index_col=None, na_values=['NA'])
        dataresult = pd.merge(dataresult,datewind(partdata, coastnumbername.get(point)),how ='outer',on='唯一标示')
        
        count = count + 1
        finished = count/len(coastpointnumber)
        print ('现在已完成：' + str(finished))

dataresult2 = dataresult.drop_duplicates('唯一标示')
dataresult2.to_excel(r'全国降水汇总(修正版).xlsx', sheet_name='result', index=False) #暂时还没做
print (dataresult)

In [ ]:
#在上述降水量数据完成之后，在excel中进行分类汇总
#以下代码则是对分类汇总的结果分成季节，各形成一个工作表
def seasons(x): 
    x = int(str(x)[-2:])
    month = range (1,13)
    Season = ['冬'] * 2+ ['春'] *3 + ['夏'] *3 +['秋'] * 3 +['冬']*1
    Dic = dict(zip(month,Season))
    return Dic.get (x)


windspeeddata = pd.read_excel('全国降水汇总结果(修正版).xlsx', index_col=None, na_values=['NA'])
Season = windspeeddata.loc[:,'唯一标示'].map(lambda x: seasons(x))
windspeeddata['季节'] = Season

spring = windspeeddata.loc[windspeeddata["季节"] == '春']
summer = windspeeddata.loc[windspeeddata["季节"] == '夏']
autumn = windspeeddata.loc[windspeeddata["季节"] == '秋']
winter = windspeeddata.loc[windspeeddata["季节"] == '冬']


datawriter = pd.ExcelWriter('全国降水结果(季节)(修正版).xlsx')
windspeeddata.to_excel(datawriter,index=False, sheet_name='Overall')
spring.to_excel(datawriter,index=False, sheet_name='spring')
summer.to_excel(datawriter,index=False, sheet_name='summer')
autumn.to_excel(datawriter,index=False, sheet_name='autumn')
winter.to_excel(datawriter,index=False, sheet_name='winter')
datawriter.save()
